In [103]:
import sys
import os
from importlib import reload
import models.vgglike as vl
import generators.csvgen as cg
import utils
reload(vl)
reload(utils)
reload(cg)

<module 'generators.csvgen' from '/home/brook/kaggle/planet/repo/generators/csvgen.py'>

###  CONFIG

In [104]:
# USE THESE SETTINGS FOR A REALISTIC SAMPLE
KG_ROOT='/home/brook/kaggle/planet'
DATA_ROOT=os.environ.get('DATA')
DATA_DIR='planet'
IMG_TYPE='tif'

In [105]:
ROOT=f'{DATA_ROOT}/{DATA_DIR}'
JPG_DIR = os.path.join(ROOT, 'train-jpg')
TIF_DIR = os.path.join(ROOT, 'train-tif')

In [106]:
if IMG_TYPE=='tif':
    BANDS=4
    IMG_DIR=TIF_DIR
else:
    BANDS=3
    IMG_DIR=JPG_DIR

In [107]:
TRAIN_SIZE=200
VALID_SIZE=50
BATCH_SIZE=32
CSV_DIR=f'{KG_ROOT}/play'
TRAIN_CSV=f'{CSV_DIR}/train-sample.csv'
VALID_CSV=f'{CSV_DIR}/valid-sample.csv'

### GENERATORS

In [108]:
train=cg.CSVGen(TRAIN_CSV,image_dir=IMG_DIR,batch_size=BATCH_SIZE)
valid=cg.CSVGen(VALID_CSV,image_dir=IMG_DIR,batch_size=BATCH_SIZE)

In [109]:
imgs=next(valid.data())[0]
lbls=next(valid.data())[1]
print(imgs.shape)
print(lbls.shape)

(32, 256, 256, 4)
(32, 17)


### DUMMY MODEL

In [110]:
dvgg=vl.DummyVGG(batch_input_shape=(None,256, 256,BANDS))

In [111]:
dvgg.model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_8 (Batch (None, 256, 256, 4)       16        
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 258, 258, 4)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 256, 256, 32)      1184      
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 258, 258, 32)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 256, 256, 32)      9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 524288)            0         
__________

In [113]:
dvgg.fit_gen(TRAIN_SIZE,VALID_SIZE,6,
             train_gen=train.data(),valid_gen=valid.data(),sample_pct=0.4)

Epoch 1/6
13/13 [==============================] - 53s - loss: 1.6197 - acc: 0.8601 - val_loss: 1.5718 - val_acc: 0.9022